In [1]:
from scipy.io import loadmat

In [2]:
data_folder = '/home/users/ghwilson/../../../oak/stanford/groups/shenoy/cortex/data/Jenkins/array/2009-11-23/js_11_23_09.mat'

In [3]:
import numpy as np

In [4]:
jenkins_data = loadmat(data_folder)